# Content based recommender system

In this notebook you will learn how to build a content based recommender-system...

requirements:

In [82]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

preprosessed df from [here](https://github.com/ruthgn/Beer-Recommender):

In [40]:

df = pd.read_csv("data/updated_beer_profile_and_ratings.csv")
df.head()

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96


In [33]:
df.shape

(3197, 25)

ratebeer df from [here](https://www.kaggle.com/dfsets/ehallmar/beers-breweries-and-beer-reviews?select=reviews.csv):

In [34]:
df_rb = pd.read_csv("data_ratebeer/reviews.csv")
df_rb.head()

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.00,4.00,4.00,4.25,4.00,4.03
1,125646,_dirty_,2017-12-21,,4.50,4.50,4.50,4.50,4.50,4.50
2,125646,CJDUBYA,2017-12-21,,4.75,4.75,4.75,4.75,4.75,4.75
3,125646,GratefulBeerGuy,2017-12-20,0% 16 oz can. Funny story: As I finally wal...,4.75,4.75,4.50,4.50,4.50,4.58
4,125646,LukeGude,2017-12-20,Classic TH NEIPA. Overflowing head and bouq...,4.25,4.50,4.25,4.25,4.25,4.31


In [35]:
df_rb["text"][:5]

0       750 ml bottle, 2016 vintage, bottle #304 of...
1                                                     
2                                                     
3       0% 16 oz can. Funny story: As I finally wal...
4       Classic TH NEIPA. Overflowing head and bouq...
Name: text, dtype: object

In [36]:
df_rb["text"].iloc[0]

"\xa0\xa0 750 ml bottle, 2016 vintage, bottle #304 of 360. Served in a DFH snifter, the beer pours dark brown/black with about an inch tan head that stuck around a little while. There's also a good amount of lacing. I like the aroma, the brew smells like dark fruit (raisin, plum), chocolate, roasted malt, and a bit of licorice. For the most part I think the taste is just like the aroma, but with some coffee flavor noticeable too. Mouthfeel/body is medium/full, it's creamy and coating with a moderate amount of carbonation. I think it's a good overall brew, worth picking up if you want a nice straight forward imperial stout. Here's hoping Four String brings this back in the future! $9.99 a bottle. \xa0"

In [37]:
df_rb.shape

(9073128, 10)

The ratebeer df contains, apart from beer characteristics, user reviews in text form. Those might be useful or not. Regarding the fact that this df set has 9 Mio entries, I will stay with the beer advocate dfset for now.

## 1. data cleaning and EDA

In [47]:
df.shape 

(3197, 25)

In [48]:
df.describe()

,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,Bitter,Sweet,Sour,Salty,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
count,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000,3197.000000
mean,6.526688,21.180482,38.986863,16.515796,46.129496,17.055990,36.364404,58.270879,33.145449,1.017204,38.529559,40.924617,18.345637,75.330935,3.638789,3.754393,3.660428,3.702496,3.749690,233.284955
std,2.546997,13.242242,21.355281,10.410661,25.947842,17.331334,25.791152,34.281310,35.780172,2.132651,32.296646,30.403641,23.756582,39.909338,0.503209,0.403416,0.449937,0.510361,0.387217,361.811847
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.509615,1.571429,1.285714,1.214286,1.471154,1.000000
25%,5.000000,15.000000,25.000000,9.000000,29.000000,6.000000,17.000000,33.000000,11.000000,0.000000,12.000000,18.000000,4.000000,45.000000,3.422559,3.604651,3.470021,3.500000,3.622449,23.000000
50%,6.000000,20.000000,35.000000,14.000000,40.000000,11.000000,31.000000,54.000000,22.000000,0.000000,29.000000,33.000000,10.000000,73.000000,3.720183,3.833333,3.741667,3.791667,3.825397,93.000000
75%,7.600000,25.000000,45.000000,21.000000,58.000000,22.000000,52.000000,77.000000,42.000000,1.000000,60.000000,56.000000,23.000000,103.000000,3.978000,4.000000,3.965587,4.033333,3.995192,284.000000
max,57.500000,65.000000,100.000000,81.000000,175.000000,139.000000,150.000000,263.000000,284.000000,48.000000,175.000000,172.000000,184.000000,239.000000,5.000000,4.666667,5.000000,5.000000,4.634615,3290.000000


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3197 entries, 0 to 3196
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               3197 non-null   object 
 1   Style              3197 non-null   object 
 2   Brewery            3197 non-null   object 
 3   Beer Name (Full)   3197 non-null   object 
 4   Description        3197 non-null   object 
 5   ABV                3197 non-null   float64
 6   Min IBU            3197 non-null   int64  
 7   Max IBU            3197 non-null   int64  
 8   Astringency        3197 non-null   int64  
 9   Body               3197 non-null   int64  
 10  Alcohol            3197 non-null   int64  
 11  Bitter             3197 non-null   int64  
 12  Sweet              3197 non-null   int64  
 13  Sour               3197 non-null   int64  
 14  Salty              3197 non-null   int64  
 15  Fruits             3197 non-null   int64  
 16  Hoppy              3197 

In [44]:
#remove duplicates
df.drop_duplicates(inplace = True)

In [46]:
df.shape

(3197, 25)

Since the shape is still the same, the dataframe did not contain duplicates.

### Data Scaling

## 2. Simple model

In [54]:
df.head()

,Name,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
0,Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497
1,Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481
2,Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377
3,Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368
4,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96


In [68]:
#matrix = df.iloc[:,5:-1]

In [83]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['Description'])

In [84]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.00000000e+00, 3.22311393e-02, 4.62455991e-02, ...,
        2.61037018e-02, 2.61037018e-02, 5.44062446e-04],
       [3.22311393e-02, 1.00000000e+00, 1.14964677e-01, ...,
        1.11189813e-02, 1.11189813e-02, 8.46053443e-03],
       [4.62455991e-02, 1.14964677e-01, 1.00000000e+00, ...,
        1.49214249e-02, 1.49214249e-02, 3.10997536e-04],
       ...,
       [2.61037018e-02, 1.11189813e-02, 1.49214249e-02, ...,
        1.00000000e+00, 1.00000000e+00, 2.08423483e-02],
       [2.61037018e-02, 1.11189813e-02, 1.49214249e-02, ...,
        1.00000000e+00, 1.00000000e+00, 2.08423483e-02],
       [5.44062446e-04, 8.46053443e-03, 3.10997536e-04, ...,
        2.08423483e-02, 2.08423483e-02, 1.00000000e+00]])

In [85]:
idx = indices1["Amber"]
idx

Name
Amber       0
Amber    1063
dtype: int64

In [86]:
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores

[(0,
  array([1.00000000e+00, 3.22311393e-02, 4.62455991e-02, ...,
         2.61037018e-02, 2.61037018e-02, 5.44062446e-04])),
 (1,
  array([0.04672194, 0.04106711, 0.02695366, ..., 0.02116804, 0.02116804,
         0.00906622]))]

In [87]:
 sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
 sim_scores

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [88]:
# Build a 1-dimensional array with beer names
titles1 = df['Name']
indices1 = pd.Series(df.index, index=df['Name'])

# Function that get book recommendations based on the cosine similarity score of books tags
def beer_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    beer_indices = [i[0] for i in sim_scores]
    return titles.iloc[beer_indices]

In [89]:
beer_recommendations('Amber')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()